# RAG Pipeline

1. gpt4all embeddings
2. llama2; 7b params; 6 bits quanitization

# VectorDB Setup

In [ ]:
!pip3 install --upgrade --quiet  langchain langchain-community langchainhub gpt4all chromadb 
!pip3 install llama-cpp-python

In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

/Users/rohitmalhotra/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.vectorstores import Chroma

# vectorstore = Chroma.from_documents(documents=all_splits, embedding=GPT4AllEmbeddings())
vectorstore = Chroma(persist_directory="gpt4allDB", embedding_function=GPT4AllEmbeddings())


bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


In [3]:
question = "What is buggy?"
docs = vectorstore.similarity_search(question)
len(docs)

4

In [15]:
docs

[Document(page_content='April 10, 2019\n\nBuggy Races Keep Rolling at Carnegie Mellon\n\nIn its 99th year, the tradition is a Spring Carnival treat\n\nBy Heidi Opdyke\n\nSweepstakes, also known as the Buggy Races, has come a long way at Carnegie Mellon University. The slick, torpedo-like vessels carrying drivers with nerves of steel are a far cry from the two-man teams that once changed places halfway through a race and rode in everything from rain barrels with bicycle wheels to three-wheeled ash cans 99 years ago.', metadata={'source': 'Data/Buggy News/02.txt'}),
 Document(page_content='Sweepstakes Slang\n\nBuggy: Vehicle being raced and also a nickname for the competition.\n\nChute: A section of the freeroll portion of the buggy course (near the southwestern end of Frew Street at its intersection with Schenley Drive) where buggies make the sharp righthand turn from Schenley Drive onto Frew Street.\n\nChute Flagger: Team member who provides a signal for buggy drivers to know when to s

# Chat Agent Setup

In [ ]:
!pip install --upgrade --quiet  llama-cpp-python
!wget https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q6_K.gguf

In [ ]:
from langchain_community.llms import LlamaCpp


n_gpu_layers = 1  # Metal set to 1 is enough.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="llama-2-7b-chat.Q6_K.gguf",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=2048,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    verbose=True,
)


# RAG

In [ ]:
from langchain import hub
from langchain_core.runnables import RunnablePassthrough, RunnablePick
from langchain_core.prompts.chat import HumanMessagePromptTemplate, PromptTemplate

rag_prompt = hub.pull("rlm/rag-prompt")
# rag_prompt.messages
prompt = HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use two sentences maximum and keep the answer CONCISE. Keep the answer CONCISE.\nQuestion: {question} \nContext: {context} \nAnswer:"))
rag_prompt.messages = [prompt]

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

# Chain
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


retriever = vectorstore.as_retriever()
qa_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)



In [ ]:
question = "What are the approaches to Task Decomposition?"
response = qa_chain.invoke(question)

In [ ]:
f = open("SubmissionData/test/questions.txt", "r")
questions = f.readlines()
f.close()

questions = [i.strip() for i in questions]

In [ ]:
from tqdm import tqdm

answers = []
for i in tqdm(range(len(questions))):
  response = qa_chain.invoke(questions[i])
  answers.append(response)